![Vespa logo](https://vespa.ai/assets/vespa-logo-color.png)

# Exchange data with applications

This notebook demonstrates ways to feed, get, update and delete data.

See end of this notebook to use the Vespa CLI for high-throughput feeding, instead of using pyvespa functions.

[Install pyvespa](https://pyvespa.readthedocs.io/) and start Docker, validate minimum 4G available:

In [ ]:
!docker info | grep "Total Memory"

Deploy a sample test application:

In [4]:
import os
from vespa.package import (
    Document,
    Field,
    Schema,
    ApplicationPackage,
)
from vespa.deployment import VespaDocker

class TestApp(ApplicationPackage):
    def __init__(self, name: str = "testapp"):
        context_document = Document(
            fields=[
                Field(name="questions",  type="array<int>", indexing=["summary", "attribute"]),
                Field(name="dataset",    type="string",     indexing=["summary", "attribute"]),
                Field(name="context_id", type="int",        indexing=["summary", "attribute"]),
                Field(name="text",       type="string",     indexing=["summary", "index"])
            ]
        )
        context_schema = Schema(
            name="context",
            document=context_document,
        )
        sentence_document = Document(
            inherits="context",
            fields=[
                Field(name="sentence_embedding", type="tensor<float>(x[512])", indexing=["attribute"])
            ],
        )
        sentence_schema = Schema(
            name="sentence",
            document=sentence_document,
        )
        super().__init__(
            name=name,
            schema=[context_schema, sentence_schema],
        )

app_package = TestApp()
vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Waiting for application status, 10/300 seconds...
Waiting for application status, 15/300 seconds...
Waiting for application status, 20/300 seconds...
Waiting for application status, 25/300 seconds...
Finished deployment.


Download sample data:

In [5]:
import json, requests

sentence_data = json.loads(
    requests.get("https://data.vespa.oath.cloud/blog/qa/sample_sentence_data_100.json").text
)
list(sentence_data[0].keys())

['text', 'dataset', 'questions', 'context_id', 'sentence_embedding']

## Feed data

Feed data in a batch or one document at a time.

### Batch

Prepare the data as a list of dicts having the `id` key holding a unique id of the data point
and the `fields` key holding a dict with the data fields:

In [7]:
batch_feed = [
    {
        "id": idx, 
        "fields": sentence
    }
    for idx, sentence in enumerate(sentence_data)
]

Feed the batch using [feed_batch](reference-api.rst#vespa.application.Vespa.feed_batch):

In [8]:
response = app.feed_batch(schema="sentence", batch=batch_feed)

Successful documents fed: 100/100.
Batch progress: 1/1.


### Individual data points
#### Synchronous
Syncronously feeding individual data points is similar to batch feeding, except that you have more control when looping through your dataset:

In [9]:
response = []
for idx, sentence in enumerate(sentence_data):
    response.append(
        app.feed_data_point(schema="sentence", data_id=idx, fields=sentence)
    )

#### Asynchronous
`app.asyncio()` returns a `VespaAsync` instance that contains async operations such as `feed_data_point`.
Using the `async with` context manager ensures that we open and close the appropriate connections
required for async feeding:

In [10]:
async with app.asyncio() as async_app:
    response = await async_app.feed_data_point(
        schema="sentence",
        data_id=idx,
        fields=sentence,
    )

We can then use asyncio constructs like `create_task` and `wait` to create different types of asynchronous flows:

In [11]:
from asyncio import create_task, wait, ALL_COMPLETED

async with app.asyncio() as async_app:
    feed = []
    for idx, sentence in enumerate(sentence_data):
        feed.append(
            create_task(
                async_app.feed_data_point(
                    schema="sentence",
                    data_id=idx,
                    fields=sentence,
                )
            )
        )
    await wait(feed, return_when=ALL_COMPLETED)
    response = [x.result() for x in feed]

<div class="alert alert-info">

**Note**: The code above runs from a Jupyter Notebook
because it already has its async event loop running in the background.
You must create your event loop when running this code on an environment without one,
just like any asyncio code requires.
</div>

## Get data
Similarly to the examples about feeding, we can get a batch of data or get individual data points.

### Batch
Prepare the data as a list of dicts having the `id` key holding a unique id of the data point.
We then get the batch from the desired schema using the
[get_batch](reference-api.rst#vespa.application.Vespa.get_batch) method.

In [12]:
batch = [{"id": idx} for idx, sentence in enumerate(sentence_data)]
response = app.get_batch(schema="sentence", batch=batch)

### Individual data points
We can get individual data points synchronously or asynchronously.

#### Synchronous

In [13]:
response = app.get_data(schema="sentence", data_id=0)

#### Asynchronous

In [14]:
async with app.asyncio() as async_app:
    response = await async_app.get_data(schema="sentence",data_id=0)

<div class="alert alert-info">

**Note**: The code above runs from a Jupyter Notebook because it already has its async event loop running in the background. You must create your event loop when running this code on an environment without one, just like any asyncio code requires.
</div>

## Update data
Similarly to the examples about feeding, we can update a batch of data or update individual data points.

### Batch
Prepare the data as a list of dicts having the `id` key holding a unique id of the data point,
the `fields` key holding a dict with the fields to be updated
and an optional `create` key with a boolean value to indicate if a data point should be created
in case it does not exist (default to `False`):

In [15]:
batch_update = [
    {
        "id": idx,           # data_id
        "fields": sentence,  # fields to be updated
        "create": True       # Optional. Create data point if not exist, default to False.
        
    }
    for idx, sentence in enumerate(sentence_data)
]

We then update the batch using [update_batch](reference-api.rst#vespa.application.Vespa.update_batch):

In [16]:
response = app.update_batch(schema="sentence", batch=batch_update)

### Individual data points
We can update individual data points synchronously or asynchronously.

#### Synchronous

In [17]:
response = app.update_data(schema="sentence", data_id=0, fields=sentence_data[0], create=True)

#### Asynchronous

In [18]:
async with app.asyncio() as async_app:
    response = await async_app.update_data(schema="sentence",data_id=0, fields=sentence_data[0], create=True)

<div class="alert alert-info">

**Note**: The code above runs from a Jupyter Notebook because it already has its async event loop running in the background. You must create your event loop when running this code on an environment without one, just like any asyncio code requires.
</div>

## Delete data
Similarly to the examples about feeding, we can delete a batch of data or delete individual data points.

### Batch
Prepare the data as a list of dicts having the `id` key holding a unique id of the data point.
We then delete the batch from the desired schema using the
[delete_batch](reference-api.rst#vespa.application.Vespa.delete_batch) method.

In [19]:
batch = [{"id": idx} for idx, sentence in enumerate(sentence_data)]
response = app.delete_batch(schema="sentence", batch=batch)

### Individual data points
We can delete individual data points synchronously or asynchronously.

#### Synchronous

In [20]:
response = app.delete_data(schema="sentence", data_id=0)

#### Asynchronous

In [21]:
async with app.asyncio() as async_app:
    response = await async_app.delete_data(schema="sentence",data_id=0)

<div class="alert alert-info">

**Note**: The code above runs from a Jupyter Notebook because it already has its async event loop running in the background. You must create your event loop when running this code on an environment without one, just like any asyncio code requires.
</div>

## Feed using Vespa CLI

Pyvespa's feeding functions above are not optimised for performance, with little error handling.
For large data sets, a better aternative is to use the [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html),
the Vespa Command-Line Interface: Export a feed-file, and feed using the `vespa`-utility:

In [25]:
!brew install vespa-cli

==> Downloading https://formulae.brew.sh/api/formula.jws.json
#=#=#                                                                          
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
To reinstall 8.198.18, run:
  brew reinstall vespa-cli


In [24]:
import pandas
from vespa.application import df_to_vespafeed

df = pandas.DataFrame({
    "context_id": [0, 1, 2],
    "text": ["text 1", "text 2", "text 3"]
})
with open("feed.json", "w") as f:
    f.write(df_to_vespafeed(df, "sentence", "context_id"))
    
!vespa feed feed.json

{
  "feeder.seconds": 0.073,
  "feeder.ok.count": 3,
  "feeder.ok.rate": 3.000,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 3,
  "http.request.bytes": 138,
  "http.request.MBps": 0.000,
  "http.exception.count": 0,
  "http.response.count": 3,
  "http.response.bytes": 246,
  "http.response.MBps": 0.000,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 51,
  "http.response.latency.millis.avg": 58,
  "http.response.latency.millis.max": 72,
  "http.response.code.counts": {
    "200": 3
  }
}


Note that each record needs a field that can be used as a unique id.

## Cleanup

In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()